参考 http://www.cnblogs.com/zyly/p/9146787.html

In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
from tensorflow.contrib.slim.nets import vgg

IMAGE_SIZE = vgg.vgg_16.default_image_size
print(IMAGE_SIZE)

batch_size = 64

224


### 导入数据

In [2]:
import os
import numpy as np
import cv2
import time

def input_data(npz_file):
    if os.path.exists(npz_file) :
        bird_data = np.load(npz_file)
        return bird_data['train_img'],bird_data['test_img'],bird_data['train_label'],bird_data['test_label']
    else:      
        data_path = os.path.join('../../..','data','CUB_200_2011')
        print(os.listdir(data_path))

        train_test_split_file = os.path.join(data_path,'train_test_split.txt')
        with open(train_test_split_file,'r') as file:
            train_test_split = np.array([i.split()[1] for i in file.readlines()]).astype('bool')
        print(train_test_split,train_test_split.size)

        img_paths_file = os.path.join(data_path,'images.txt')
        with open(img_paths_file,'r') as file:
            img_paths = [i.split()[1] for i in file.readlines()]
        print(img_paths[:1],len(img_paths))

        img_labels_file = os.path.join(data_path,'image_class_labels.txt')
        with open(img_labels_file,'r') as file:
            img_labels = np.array([i.split()[1] for i in file.readlines()]).astype('int')
        print(img_labels,len(img_labels))

        img_dir = os.path.join(data_path,'images')

        img_paths_train = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if train_test_split[i]]
        print(img_paths_train[:1],len(img_paths_train))
        img_paths_test = [os.path.join(img_dir,os.path.sep.join(path.split('/'))) for i,path in enumerate(img_paths) if not train_test_split[i]]
        print(img_paths_test[:1],len(img_paths_test))

        train_img = np.array([cv2.resize(cv2.imread(i),(224,224)) for i in img_paths_train])
        test_img = np.array([cv2.resize(cv2.imread(i),(224,224)) for i in img_paths_test])
        train_label = np.array([l for i,l in enumerate(img_labels) if train_test_split[i] ])
        test_label = np.array([l for i,l in enumerate(img_labels) if not train_test_split[i]])
        print(train_label,train_label.size)
        print(test_label,test_label.size)

        np.savez(npz_file,train_img=train_img,test_img=test_img,train_label=train_label,test_label=test_label)
        return train_img,test_img,train_label,test_label
    
x_train,x_test,y_train,y_test = input_data('bird_data_224.npz')


num_classes = 200

# 数据预处理，把 0-255的灰度值转成 0-1 之间的浮点数
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255
y_train, y_test = np.array(y_train)-1, np.array(y_test)-1



print('type:',type(x_train),type(y_train))
print('shape:',x_train.shape,y_train.shape)
print('size:',x_train.size,y_train.size)

type: <class 'numpy.ndarray'> <class 'numpy.ndarray'>
shape: (5994, 224, 224, 3) (5994,)
size: 902264832 5994


### 批量数据 生成器

In [3]:
def shuffle_aligned_list(data):
    """Shuffle arrays in a list by shuffling each array identically."""
    num = data[0].shape[0]
    p = np.random.permutation(num)
    return [d[p] for d in data]

def batch_generator(data, batch_size, shuffle=True):
    """Generate batches of data.

    Given a list of array-like objects, generate batches of a given
    size by yielding a list of array-like objects corresponding to the
    same slice of each input.
    """
    if shuffle:
        data = shuffle_aligned_list(data)

    batch_count = 0
    while True:
        if batch_count * batch_size + batch_size > len(data[0]):
            batch_count = 0

            if shuffle:
                data = shuffle_aligned_list(data)

        start = batch_count * batch_size
        end = start + batch_size
        batch_count += 1
        yield [d[start:end] for d in data]
        
data_gen_test = batch_generator([x_test,y_test], batch_size, shuffle=True)

from keras.preprocessing.image import ImageDataGenerator
 
data_gen_train = ImageDataGenerator(
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    rotation_range=30,    #0-180
    horizontal_flip = True)


Using TensorFlow backend.


### 微调 VGG

首先，单独训练 fc8，训练了 196 epochs

然后，训练整个网络，训练了 20 epochs

In [4]:
training_epochs = 1000

learning_rate = 1e-4
display_epoch = 1
'''
演示一个VGG16的例子 
微调 这里只调整VGG16最后一层全连接层，把1000类改为5类 
对网络进行训练   使用slim库简化代码
'''    
def loss(logits, labels):

    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits, labels=labels, name='cross_entropy_per_example')

    return tf.reduce_mean(cross_entropy, name='cross_entropy')


#用于保存微调后的检查点文件和日志文件路径
train_log_dir = './vgg_16_2016_08_28/slim_fine_tune'
train_log_file = 'birds_fine_tune.ckpt'

#官方下载的检查点文件路径
checkpoint_file = './vgg_16_2016_08_28/vgg_16.ckpt'

if not tf.gfile.Exists(train_log_dir):
    tf.gfile.MakeDirs(train_log_dir)

#创建一个图，作为当前图
with tf.Graph().as_default():

    #加载数据
    image_holder = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, 3])
    label_holder = tf.placeholder(tf.int32, [None,]) 
    is_training = tf.placeholder(dtype = tf.bool)


    #创建vgg16网络  如果想冻结所有层，可以指定slim.conv2d中的 trainable=False
    logits,end_points =  vgg.vgg_16(image_holder, is_training=is_training, num_classes = num_classes)  
    print(logits.shape)

    # 获取fc8初始化函数
    fc8_variables = tf.contrib.framework.get_variables('vgg_16/fc8')
    # Restore only the convolutional layers: 从检查点载入当前图除了fc8层之外所有变量的参数
    params = slim.get_variables_to_restore(exclude=['vgg_16/fc8'])
    #用于恢复模型  如果使用这个保存或者恢复的话，只会保存或者恢复指定的变量
    restorer = tf.train.Saver(params) 

    #预测标签
    pred = tf.cast(tf.argmax(logits,axis=1),tf.int32)

    '''
    定义代价函数和优化器
    '''   

    #代价函数
    cost = loss(logits, label_holder)
  
    #设置优化器
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost,var_list=fc8_variables)
    full_optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    #预测结果评估        
    correct = tf.equal(pred, label_holder)                    #返回一个数组 表示统计预测正确或者错误 
    accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))                #求准确率

    num_batch = int(np.ceil(x_train.shape[0] / batch_size))

    #用于保存检查点文件 
    save = tf.train.Saver(max_to_keep=1) 

    #恢复模型
    with tf.Session() as sess:      
        sess.run(tf.global_variables_initializer())

        #检查最近的检查点文件
        ckpt = tf.train.latest_checkpoint(train_log_dir)
        if ckpt != None:
            save.restore(sess,ckpt)
            print('从上次训练保存后的模型继续训练！')
        else:
            restorer.restore(sess, checkpoint_file)                
            print('从官方模型加载训练！')



        print('开始训练！')
        for epoch in range(training_epochs):

            total_cost = 0.0                
            for i in range(num_batch):
                image_batch, label_batch = next(data_gen_train.flow(x_train, y_train, batch_size=batch_size))                                               
                _,loss,acc = sess.run([full_optimizer,cost,accuracy],feed_dict={image_holder:image_batch,label_holder:label_batch,is_training:True})
                total_cost += loss
                print('epoch: {} batch: {} train batch accuracy: {} loss: {}'.format(epoch, i, acc, loss))
                

            #打印信息
            if epoch % display_epoch == 0:
                print('Epoch {}/{}  average cost {:.9f}'.format(epoch+1,training_epochs,total_cost/num_batch))

            #进行预测处理
            image_batch, label_batch = next(data_gen_test)                                                                 
            cost_values,accuracy_value = sess.run([cost,accuracy],feed_dict = {image_holder:image_batch,label_holder:label_batch,is_training:False})
            print('Epoch {}/{}  Test cost {:.9f}'.format(epoch+1,training_epochs,cost_values))
            print('准确率:',accuracy_value)


            #保存模型
            save.save(sess,os.path.join(train_log_dir,train_log_file),global_step = epoch)
            print('Epoch {}/{}  模型保存成功'.format(epoch+1,training_epochs))

            
        print('训练完成')


(?, 200)
INFO:tensorflow:Restoring parameters from ./vgg_16_2016_08_28/slim_fine_tune\birds_fine_tune.ckpt-196
从上次训练保存后的模型继续训练！
开始训练！
epoch: 0 batch: 0 train batch accuracy: 0.078125 loss: 4.278685569763184
epoch: 0 batch: 1 train batch accuracy: 0.0 loss: 12.081350326538086
epoch: 0 batch: 2 train batch accuracy: 0.0 loss: 11.747150421142578
epoch: 0 batch: 3 train batch accuracy: 0.015625 loss: 8.02389144897461
epoch: 0 batch: 4 train batch accuracy: 0.015625 loss: 6.392911434173584
epoch: 0 batch: 5 train batch accuracy: 0.0 loss: 6.545704364776611
epoch: 0 batch: 6 train batch accuracy: 0.0 loss: 6.41831111907959
epoch: 0 batch: 7 train batch accuracy: 0.015625 loss: 5.695311069488525
epoch: 0 batch: 8 train batch accuracy: 0.0 loss: 5.759634017944336
epoch: 0 batch: 9 train batch accuracy: 0.03125 loss: 5.439011096954346
epoch: 0 batch: 10 train batch accuracy: 0.015625 loss: 5.738134384155273
epoch: 0 batch: 11 train batch accuracy: 0.0 loss: 5.372401237487793
epoch: 0 batch: 12 

epoch: 1 batch: 17 train batch accuracy: 0.0625 loss: 4.156362533569336
epoch: 1 batch: 18 train batch accuracy: 0.09375 loss: 4.019362926483154
epoch: 1 batch: 19 train batch accuracy: 0.078125 loss: 3.889273166656494
epoch: 1 batch: 20 train batch accuracy: 0.078125 loss: 4.165731906890869
epoch: 1 batch: 21 train batch accuracy: 0.125 loss: 4.04695463180542
epoch: 1 batch: 22 train batch accuracy: 0.09375 loss: 4.077465057373047
epoch: 1 batch: 23 train batch accuracy: 0.015625 loss: 4.227624893188477
epoch: 1 batch: 24 train batch accuracy: 0.109375 loss: 4.00766658782959
epoch: 1 batch: 25 train batch accuracy: 0.0625 loss: 4.070629596710205
epoch: 1 batch: 26 train batch accuracy: 0.09375 loss: 4.043320655822754
epoch: 1 batch: 27 train batch accuracy: 0.109375 loss: 3.903799533843994
epoch: 1 batch: 28 train batch accuracy: 0.109375 loss: 3.889280080795288
epoch: 1 batch: 29 train batch accuracy: 0.078125 loss: 4.256725311279297
epoch: 1 batch: 30 train batch accuracy: 0.0625 lo

epoch: 2 batch: 34 train batch accuracy: 0.296875 loss: 2.580561637878418
epoch: 2 batch: 35 train batch accuracy: 0.40625 loss: 2.4681506156921387
epoch: 2 batch: 36 train batch accuracy: 0.234375 loss: 3.0198559761047363
epoch: 2 batch: 37 train batch accuracy: 0.21875 loss: 2.9742815494537354
epoch: 2 batch: 38 train batch accuracy: 0.1875 loss: 2.9190540313720703
epoch: 2 batch: 39 train batch accuracy: 0.375 loss: 2.3026413917541504
epoch: 2 batch: 40 train batch accuracy: 0.25 loss: 2.9377810955047607
epoch: 2 batch: 41 train batch accuracy: 0.28125 loss: 2.5895156860351562
epoch: 2 batch: 42 train batch accuracy: 0.34375 loss: 2.7876720428466797
epoch: 2 batch: 43 train batch accuracy: 0.28125 loss: 3.0213284492492676
epoch: 2 batch: 44 train batch accuracy: 0.28125 loss: 2.869292736053467
epoch: 2 batch: 45 train batch accuracy: 0.3125 loss: 2.435699939727783
epoch: 2 batch: 46 train batch accuracy: 0.265625 loss: 2.8691935539245605
epoch: 2 batch: 47 train batch accuracy: 0.28

epoch: 3 batch: 50 train batch accuracy: 0.5 loss: 1.9927890300750732
epoch: 3 batch: 51 train batch accuracy: 0.3125 loss: 2.2092509269714355
epoch: 3 batch: 52 train batch accuracy: 0.390625 loss: 2.373884677886963
epoch: 3 batch: 53 train batch accuracy: 0.4375 loss: 2.0283703804016113
epoch: 3 batch: 54 train batch accuracy: 0.515625 loss: 1.7471423149108887
epoch: 3 batch: 55 train batch accuracy: 0.453125 loss: 2.0578789710998535
epoch: 3 batch: 56 train batch accuracy: 0.46875 loss: 2.176562786102295
epoch: 3 batch: 57 train batch accuracy: 0.484375 loss: 2.095160484313965
epoch: 3 batch: 58 train batch accuracy: 0.421875 loss: 2.419982433319092
epoch: 3 batch: 59 train batch accuracy: 0.390625 loss: 2.3381826877593994
epoch: 3 batch: 60 train batch accuracy: 0.453125 loss: 1.9605984687805176
epoch: 3 batch: 61 train batch accuracy: 0.484375 loss: 1.9472193717956543
epoch: 3 batch: 62 train batch accuracy: 0.515625 loss: 1.8691887855529785
epoch: 3 batch: 63 train batch accuracy

epoch: 4 batch: 66 train batch accuracy: 0.515625 loss: 1.6384446620941162
epoch: 4 batch: 67 train batch accuracy: 0.609375 loss: 1.4084529876708984
epoch: 4 batch: 68 train batch accuracy: 0.515625 loss: 1.6832406520843506
epoch: 4 batch: 69 train batch accuracy: 0.625 loss: 1.4368840456008911
epoch: 4 batch: 70 train batch accuracy: 0.46875 loss: 1.9572594165802002
epoch: 4 batch: 71 train batch accuracy: 0.46875 loss: 1.7105238437652588
epoch: 4 batch: 72 train batch accuracy: 0.40625 loss: 1.8386842012405396
epoch: 4 batch: 73 train batch accuracy: 0.59375 loss: 1.5296895503997803
epoch: 4 batch: 74 train batch accuracy: 0.5 loss: 1.7907350063323975
epoch: 4 batch: 75 train batch accuracy: 0.59375 loss: 1.2831971645355225
epoch: 4 batch: 76 train batch accuracy: 0.484375 loss: 1.64376699924469
epoch: 4 batch: 77 train batch accuracy: 0.5 loss: 1.5297796726226807
epoch: 4 batch: 78 train batch accuracy: 0.484375 loss: 1.9130933284759521
epoch: 4 batch: 79 train batch accuracy: 0.60

...

epoch: 17 batch: 38 train batch accuracy: 0.9375 loss: 0.22011306881904602
epoch: 17 batch: 39 train batch accuracy: 0.828125 loss: 0.5509326457977295
epoch: 17 batch: 40 train batch accuracy: 0.875 loss: 0.4737092852592468
epoch: 17 batch: 41 train batch accuracy: 0.875 loss: 0.3262169361114502
epoch: 17 batch: 42 train batch accuracy: 0.9375 loss: 0.1449447125196457
epoch: 17 batch: 43 train batch accuracy: 0.90625 loss: 0.41324907541275024
epoch: 17 batch: 44 train batch accuracy: 0.9375 loss: 0.1411159783601761
epoch: 17 batch: 45 train batch accuracy: 0.875 loss: 0.3061140179634094
epoch: 17 batch: 46 train batch accuracy: 0.90625 loss: 0.2805890440940857
epoch: 17 batch: 47 train batch accuracy: 0.890625 loss: 0.3647463321685791
epoch: 17 batch: 48 train batch accuracy: 0.890625 loss: 0.3292909860610962
epoch: 17 batch: 49 train batch accuracy: 0.890625 loss: 0.29342013597488403
epoch: 17 batch: 50 train batch accuracy: 0.953125 loss: 0.2297920286655426
epoch: 17 batch: 51 train 

epoch: 18 batch: 52 train batch accuracy: 0.90625 loss: 0.3631214499473572
epoch: 18 batch: 53 train batch accuracy: 0.921875 loss: 0.2246159315109253
epoch: 18 batch: 54 train batch accuracy: 0.953125 loss: 0.26360368728637695
epoch: 18 batch: 55 train batch accuracy: 0.921875 loss: 0.3040531873703003
epoch: 18 batch: 56 train batch accuracy: 0.9375 loss: 0.21070387959480286
epoch: 18 batch: 57 train batch accuracy: 0.921875 loss: 0.3034963011741638
epoch: 18 batch: 58 train batch accuracy: 0.953125 loss: 0.24590972065925598
epoch: 18 batch: 59 train batch accuracy: 0.84375 loss: 0.6230030655860901
epoch: 18 batch: 60 train batch accuracy: 0.890625 loss: 0.36760279536247253
epoch: 18 batch: 61 train batch accuracy: 0.90625 loss: 0.26192182302474976
epoch: 18 batch: 62 train batch accuracy: 0.921875 loss: 0.1883954405784607
epoch: 18 batch: 63 train batch accuracy: 0.953125 loss: 0.22242942452430725
epoch: 18 batch: 64 train batch accuracy: 0.890625 loss: 0.24600878357887268
epoch: 18 

epoch: 19 batch: 66 train batch accuracy: 0.875 loss: 0.40202945470809937
epoch: 19 batch: 67 train batch accuracy: 0.90625 loss: 0.34166252613067627
epoch: 19 batch: 68 train batch accuracy: 0.9375 loss: 0.1798204928636551
epoch: 19 batch: 69 train batch accuracy: 0.921875 loss: 0.19530189037322998
epoch: 19 batch: 70 train batch accuracy: 0.859375 loss: 0.3420848250389099
epoch: 19 batch: 71 train batch accuracy: 0.953125 loss: 0.17470404505729675
epoch: 19 batch: 72 train batch accuracy: 0.9375 loss: 0.2231922149658203
epoch: 19 batch: 73 train batch accuracy: 0.90625 loss: 0.4045405089855194
epoch: 19 batch: 74 train batch accuracy: 0.953125 loss: 0.18714967370033264
epoch: 19 batch: 75 train batch accuracy: 0.90625 loss: 0.31080037355422974
epoch: 19 batch: 76 train batch accuracy: 0.921875 loss: 0.274804025888443
epoch: 19 batch: 77 train batch accuracy: 0.875 loss: 0.3208746314048767
epoch: 19 batch: 78 train batch accuracy: 0.9375 loss: 0.15432360768318176
epoch: 19 batch: 79 t

epoch: 20 batch: 80 train batch accuracy: 0.90625 loss: 0.24234530329704285
epoch: 20 batch: 81 train batch accuracy: 0.96875 loss: 0.15579254925251007
epoch: 20 batch: 82 train batch accuracy: 0.90625 loss: 0.25911861658096313
epoch: 20 batch: 83 train batch accuracy: 0.90625 loss: 0.3182986378669739
epoch: 20 batch: 84 train batch accuracy: 0.890625 loss: 0.35189586877822876
epoch: 20 batch: 85 train batch accuracy: 0.953125 loss: 0.1716003715991974
epoch: 20 batch: 86 train batch accuracy: 0.859375 loss: 0.45583826303482056
epoch: 20 batch: 87 train batch accuracy: 0.9375 loss: 0.18900850415229797
epoch: 20 batch: 88 train batch accuracy: 0.859375 loss: 0.5286474823951721
epoch: 20 batch: 89 train batch accuracy: 0.9375 loss: 0.16382841765880585
epoch: 20 batch: 90 train batch accuracy: 0.921875 loss: 0.2888340950012207
epoch: 20 batch: 91 train batch accuracy: 0.953125 loss: 0.21941149234771729
epoch: 20 batch: 92 train batch accuracy: 0.984375 loss: 0.09324516355991364
epoch: 20 b

KeyboardInterrupt: 